In [2]:
!pip install BeautifulSoup4

  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
  Using cached soupsieve-2.1-py3-none-any.whl (32 kB)
You should consider upgrading via the '/Users/itversity/Projects/Internal/bootcamp/itversity-material/data-engineering-on-aws/deaws-env/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
from bs4 import BeautifulSoup as bs
import requests
def cleanup_page(url):
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    mc = soup.find('div', {'id': 'main-content'}).extract()
    for div in mc.find_all("a", {'class':'headerlink'}): 
        div.decompose()
    title = mc.find('h1').text
    for tag in mc.find_all(['div', 'span']): 
        tag.unwrap()
    for tag in mc.find_all("pre"): 
        tag.string = tag.text.replace('%%sql', '')
    for tag in mc.find_all(["script", 'h1']): 
        tag.decompose()
    mc.find('div')
    clean_html_filtered = filter(lambda e: str(e).strip() not in ('', '\n'), mc.contents)
    clean_html_list = [str(e) for e in clean_html_filtered]
    clean_html_list = filter(lambda e: str(e).strip() not in (''), str(bs('\n'.join(clean_html_list), 'html.parser')).splitlines())
    return title, str(bs('\n'.join(clean_html_list), 'html.parser'))

In [4]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

def get_landing_page_url(base_url):

    base_page = requests.get(base_url)
    base_soup = bs(base_page.content, 'html.parser')

    landing_page_url = f"{base_url}/{base_soup.find('meta')['content'].split('=')[1]}"
    return landing_page_url

In [5]:
def get_first_level_urls(landing_page_url):
    landing_page = requests.get(landing_page_url)
    landing_soup = bs(landing_page.content, 'html.parser')
    
    first_level_urls = []
    for link in landing_soup.find_all('a', {'class': 'reference internal'}):
        if link.get('href') and link.get('href') != '#':
            first_level_urls.append(link.get('href'))
    
    return first_level_urls

In [6]:
def get_all_urls(first_level_urls):
    all_urls = []
    for url in first_level_urls:
        all_urls.append(f"{base_url}/{url}")
        second_level_url = f"{base_url}/{url}"
        second_level_page = requests.get(second_level_url)
        second_level_soup = bs(second_level_page.content, 'html.parser')
        current_active = second_level_soup.find('li', {'class': 'toctree-l1 current active'})
        for link in current_active.find_all('a', {'class': 'reference internal'}):
            all_urls.append(f"{base_url}/{url.split('/')[0]}/{link.get('href')}")
    return all_urls

In [7]:
def scrape(base_url):
    landing_page_url = get_landing_page_url(base_url)
    first_level_urls = get_first_level_urls(landing_page_url)
    all_urls = get_all_urls(first_level_urls)
    return all_urls

In [8]:
base_url = 'https://python.itversity.com'
all_urls = scrape(base_url)

In [19]:
scraped_posts = []
ctr = 0
for url in all_urls:
    if ctr % 10 == 0:
        print(f'Scraped {ctr} out of {len(all_urls)}')
    title, clean_html = cleanup_page(url)
    scraped_posts.append({
        'content_url': url,
        'content_title': title,
        'contents': clean_html
    })
    ctr += 1
print(f'Scraped {ctr} out of {len(all_urls)}')

Processed 0 out of 170
Processed 10 out of 170
Processed 20 out of 170
Processed 30 out of 170
Processed 40 out of 170
Processed 50 out of 170
Processed 60 out of 170
Processed 70 out of 170
Processed 80 out of 170
Processed 90 out of 170
Processed 100 out of 170
Processed 110 out of 170
Processed 120 out of 170
Processed 130 out of 170
Processed 140 out of 170
Processed 150 out of 170
Processed 160 out of 170
Scraped 170 out of 170


In [20]:
import boto3

In [21]:
dynamodb = boto3.resource('dynamodb')

In [22]:
posts = dynamodb.Table('posts')

In [23]:
for post in scraped_posts:
    post_item = {
        'content_url': post['content_url'],
        'content_title': post['content_title'],
        'contents': post['contents']
    }
    response = posts.put_item(Item=post_item)

In [24]:
posts_all = posts.scan()

In [25]:
posts_all['Count']

171

In [26]:
posts_all['Items'][:3]

[{'contents': '<p>Let us understand how we can take care of insert into table using Python based approach.</p>\n<ul class="simple">\n<li><p>Here are the typical steps we need to follow:</p>\n<ul>\n<li><p>Make sure we have data in the form of an object or collection.</p></li>\n<li><p>Establish the connection to database.</p></li>\n<li><p>Create cursor object.</p></li>\n<li><p>Execute query statement using <code class="docutils literal notranslate">cursor.execute</code> by passing the query statement.</p></li>\n<li><p>We need to pass the object or collection as an additional argument along with query statement.</p></li>\n<li><p>We can insert one object at a time using <code class="docutils literal notranslate">execute</code> and multiple objects at a time using <code class="docutils literal notranslate">executemany</code>.</p></li>\n</ul>\n</li>\n<li><p>If you are not familiar about how to directly insert data into the table using Database level operations, you can go through <a class="r